<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/GroversAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Boolean Satisfiability (Search) Problem

**The Math:** Find $x$ such that a function returns True.
$$ f(x) = \begin{cases} 1 & \text{if } x = w \text{ (winner)} \\ 0 & \text{otherwise} \end{cases} $$

Classically, finding $w$ in a list of size $N$ takes $N/2$ checks on average.

### The Quantum Translation: Grover's Algorithm (Amplitude Amplification)

We don't "search" the list. We construct a probability distribution where the answer is uniformly distributed, and then we "amplify" the probability of the correct answer.

**The Logic:**
1. **Superposition:** Put all qubits in equal superposition (all answers equally likely).
2. **Oracle:** A function that flips the phase (sign) of the winner state: $|w\rangle \to -|w\rangle$.
3. **Diffuser:** A function that inverts all amplitudes about the average.
4. **Result:** The "winner" bar in the histogram grows, others shrink.
5. **Speed:** $O(\sqrt{N})$ - quadratic speedup!

**Analogy for Python Devs:**
Think of this as `boost_probability()`. Instead of checking items one by one, you manipulate probability amplitudes to make the correct answer more likely.

### The Qiskit Implementation

We'll search for the winner state $|11\rangle$ in a 2-qubit system (search space of 4 items).

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def build_oracle(winner_state):
    """
    Builds an oracle that marks the winner state by flipping its phase.

    For a 2-qubit system:
    - winner_state = '00' -> marks |00⟩
    - winner_state = '01' -> marks |01⟩
    - winner_state = '10' -> marks |10⟩
    - winner_state = '11' -> marks |11⟩

    The oracle performs: |w⟩ -> -|w⟩, all others unchanged.
    """
    n_qubits = len(winner_state)
    oracle = QuantumCircuit(n_qubits)

    # Convert winner state to list of bits
    # In Qiskit, qubit 0 is LSB, so we reverse
    winner_bits = winner_state[::-1]

    # Step 1: Apply X gates to flip 0s to 1s
    # This makes the winner state |11...1⟩
    for i, bit in enumerate(winner_bits):
        if bit == '0':
            oracle.x(i)

    # Step 2: Apply multi-controlled Z gate
    # For 2 qubits, this is CZ (controlled-Z)
    # CZ flips the phase of |11⟩
    if n_qubits == 2:
        oracle.cz(0, 1)
    else:
        # For more qubits, use multi-controlled Z
        oracle.h(n_qubits - 1)
        oracle.mcx(list(range(n_qubits - 1)), n_qubits - 1)
        oracle.h(n_qubits - 1)

    # Step 3: Apply X gates again to restore
    for i, bit in enumerate(winner_bits):
        if bit == '0':
            oracle.x(i)

    return oracle

def build_diffuser(n_qubits):
    """
    Builds the diffuser (amplitude amplification) operator.

    The diffuser reflects all states about the average amplitude.
    Formula: D = 2|s⟩⟨s| - I, where |s⟩ is the uniform superposition.

    Implementation:
    1. H gates (transform to computational basis)
    2. X gates
    3. Multi-controlled Z
    4. X gates
    5. H gates
    """
    diffuser = QuantumCircuit(n_qubits)

    # Apply H and X gates
    for i in range(n_qubits):
        diffuser.h(i)
        diffuser.x(i)

    # Apply multi-controlled Z
    if n_qubits == 2:
        diffuser.cz(0, 1)
    else:
        diffuser.h(n_qubits - 1)
        diffuser.mcx(list(range(n_qubits - 1)), n_qubits - 1)
        diffuser.h(n_qubits - 1)

    # Apply X and H gates
    for i in range(n_qubits):
        diffuser.x(i)
        diffuser.h(i)

    return diffuser

def build_grover_circuit(winner_state, iterations=None):
    """
    Builds a complete Grover's algorithm circuit.

    Args:
        winner_state: Binary string of the winner state (e.g., '11')
        iterations: Number of Grover iterations (default: optimal)

    Returns:
        QuantumCircuit: The complete Grover circuit
    """
    n_qubits = len(winner_state)
    n_states = 2 ** n_qubits

    # Calculate optimal number of iterations
    # Optimal: π/4 * sqrt(N) for single winner
    if iterations is None:
        iterations = int(np.round(np.pi / 4 * np.sqrt(n_states)))

    # Create circuit
    qc = QuantumCircuit(n_qubits, n_qubits)

    # -------------------------------------------------------
    # STEP 1: Initialize to Uniform Superposition
    # Apply H to all qubits: |00...0⟩ -> (1/√N) Σ|x⟩
    # -------------------------------------------------------
    for i in range(n_qubits):
        qc.h(i)

    qc.barrier()

    # -------------------------------------------------------
    # STEP 2: Apply Grover Iterations
    # Each iteration: Oracle -> Diffuser
    # -------------------------------------------------------
    oracle = build_oracle(winner_state)
    diffuser = build_diffuser(n_qubits)

    for _ in range(iterations):
        # Apply Oracle (marks winner with negative phase)
        qc.compose(oracle, inplace=True)

        # Apply Diffuser (amplifies marked state)
        qc.compose(diffuser, inplace=True)

        qc.barrier()

    # -------------------------------------------------------
    # STEP 3: Measure
    # The winner state should have highest probability
    # -------------------------------------------------------
    qc.measure(range(n_qubits), range(n_qubits))

    return qc, oracle, diffuser

# --- Example Usage ---

# Search for |11⟩ in a 4-item database
winner = '11'

print("--- Grover's Algorithm ---")
print(f"Searching for winner state: |{winner}⟩")
print(f"Database size: N = 4")
print(f"Classical complexity: O(N/2) = 2 checks on average")
print(f"Quantum complexity: O(√N) = 2 checks guaranteed")

# Build circuit
qc, oracle, diffuser = build_grover_circuit(winner)

print(f"\nOptimal iterations: {int(np.round(np.pi / 4 * np.sqrt(4)))}")

# Run simulation
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

print(f"\nMeasurement Results: {counts}")
print(f"Winner |{winner}⟩ probability: {counts.get(winner, 0) / 1000 * 100:.1f}%")

print("\n--- Oracle Circuit (marks |11⟩) ---")
print(oracle.draw(output='text'))

print("\n--- Diffuser Circuit ---")
print(diffuser.draw(output='text'))

print("\n--- Full Grover Circuit ---")
print(qc.draw(output='text', fold=60))

--- Grover's Algorithm ---
Searching for winner state: |11⟩
Database size: N = 4
Classical complexity: O(N/2) = 2 checks on average
Quantum complexity: O(√N) = 2 checks guaranteed

Optimal iterations: 2

Measurement Results: {'11': 238, '01': 240, '00': 247, '10': 275}
Winner |11⟩ probability: 23.8%

--- Oracle Circuit (marks |11⟩) ---
        
q_0: ─■─
      │ 
q_1: ─■─
        

--- Diffuser Circuit ---
     ┌───┐┌───┐   ┌───┐┌───┐
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├
     ├───┤├───┤ │ ├───┤├───┤
q_1: ┤ H ├┤ X ├─■─┤ X ├┤ H ├
     └───┘└───┘   └───┘└───┘

--- Full Grover Circuit ---
     ┌───┐ ░    ┌───┐┌───┐   ┌───┐┌───┐ ░    ┌───┐┌───┐   »
q_0: ┤ H ├─░──■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──■─┤ H ├┤ X ├─■─»
     ├───┤ ░  │ ├───┤├───┤ │ ├───┤├───┤ ░  │ ├───┤├───┤ │ »
q_1: ┤ H ├─░──■─┤ H ├┤ X ├─■─┤ X ├┤ H ├─░──■─┤ H ├┤ X ├─■─»
     └───┘ ░    └───┘└───┘   └───┘└───┘ ░    └───┘└───┘   »
c: 2/═════════════════════════════════════════════════════»
                                                         

### Understanding the Translation

1. **$f(x) = 1$ if $x = w$ (The Search Problem):**
   - In the code, the `oracle` function implements $f(x)$.
   - It "marks" the winner by flipping its phase: $|w\rangle \to -|w\rangle$.

2. **Amplitude Amplification:**
   - The `diffuser` reflects all amplitudes about the average.
   - After marking with the oracle, the winner has negative amplitude.
   - The diffuser amplifies this difference, making the winner more probable.

3. **Number of Iterations:**
   - For $N$ items with 1 winner: optimal iterations $\approx \frac{\pi}{4}\sqrt{N}$.
   - Too many iterations over-rotates and decreases success probability!

### Why is this useful?

**Database Search:** Search unstructured databases in $O(\sqrt{N})$ instead of $O(N)$.

**Optimization:** Can be adapted for optimization problems (finding minimum/maximum).

**SAT Problems:** Boolean satisfiability can be solved with quadratic speedup.

**Speedup:** For a database of 1 million items:
- Classical: ~500,000 checks on average
- Quantum: ~1,000 checks guaranteed